## KnowEnG-Research/Data_Cleanup_Pipeline Feb 27, 2017 ( Samples Clustering )
* new improved data from aws - latest

In [1]:
import os
import sys
import time

import numpy as np
import pandas as pd

import knpackage.toolbox as kn

sys.path.insert(1, '../../KnowEnG_Pipelines_Library/knpackage')
import redis_utilities as redisutil

sys.path.insert(1, '../../Data_Cleanup_Pipeline/src/')
import data_cleanup_toolbox as dc

sys.path.insert(1, '../src')
import dcp_test

In [2]:
tools_base_dir, no_more_nothing = os.path.split(os.getcwd())
tools_base_dir

'/Users/lanier4/dlanier_KnowEnG/keg_test_tools'

In [3]:
def get_samples_clustering_run_parameters(yml_dir, yml_file_name):
    """ run_parameters = get_samples_clustering_run_parameters(yml_dir, yml_file_name) """
    run_parameters = kn.get_run_parameters(yml_dir, yml_file_name)
    run_parameters['results_directory'] = kn.create_dir(os.getcwd(), 'run_dir')
    run_parameters['pipeline_type'] = 'samples_clustering_pipeline'
    run_parameters['redis_credential']['host'] = 'knowhub.org'
    run_parameters['redis_credential']['port'] = 6379
    run_parameters['redis_credential']['password'] = 'KnowEnG'
    
    return run_parameters

In [4]:
def get_spreadsheets_for_pheno(pheno_file, sp_list):
    """ spreadsheet_list = get_spreadsheets_for_pheno(pheno_file, sp_list) """
    x = pheno_file.find('.P.')
    g_str = pheno_file[:x]
    spreadsheet_list = []
    for f in sp_list:
        if f.find(g_str) >= 0: spreadsheet_list.append(f)

    return sorted(spreadsheet_list)

In [5]:
def get_spreadsheet_phenotype_dataframe(spreadsheet_data_dir, pheno_data_dir):
    """ test_result_df = get_spreadsheet_phenotype_dataframe(spreadsheet_data_dir, pheno_data_dir) """
    
    col_list = ['phenotype_file','validation_flag','message','spreadsheet_rows','spreadsheet_cols','cleanup_time']
    
    pheno_file_list = sorted(os.listdir(pheno_data_dir))
    spreadsheet_file_list = sorted(os.listdir(spreadsheet_data_dir))
    
    test_result_df = pd.DataFrame(data=np.zeros((len(spreadsheet_file_list), len(col_list))),
                                  index=spreadsheet_file_list, columns=col_list)
    
    for pheno_file in pheno_file_list:
        spreadsheet_list = get_spreadsheets_for_pheno(pheno_file, spreadsheet_file_list)
        for spreadsheet_file in spreadsheet_list:
            test_result_df.loc[spreadsheet_file, 'phenotype_file'] = pheno_file
        
    return test_result_df

In [6]:
def test_samples_clustering_cleanup(run_parameters):
    """ test_result_df = test_samples_clustering_cleanup(run_parameters) """
    
    spreadsheet_data_dir = run_parameters['spreadsheet_data_dir']
    pheno_data_dir = run_parameters['pheno_data_dir']
    
    test_result_df = get_spreadsheet_phenotype_dataframe(spreadsheet_data_dir, pheno_data_dir)
    
    for spreadsheet_file in list(test_result_df.index):
        phenotype_file = test_result_df.loc[spreadsheet_file, 'phenotype_file']
        if phenotype_file != 0:
            print(phenotype_file, spreadsheet_file)
            run_parameters['spreadsheet_name_full_path'] = os.path.join(spreadsheet_data_dir, spreadsheet_file)
            run_parameters['phenotype_full_path'] = os.path.join(pheno_data_dir, phenotype_file)
            
            tt = 0.0
            validation_flag = False
            message = "Failed to finish"

            try:
                t0 = time.time()
                #validation_flag, message = dc.run_gene_priorization_pipeline(run_parameters)
                tt = time.time() - t0
            except:
                pass
            
            test_result_df.loc[spreadsheet_file, 'message'] = message
            test_result_df.loc[spreadsheet_file, 'cleanup_time'] = tt
            test_result_df.loc[spreadsheet_file, 'validation_flag'] = validation_flag
            
    return test_result_df

In [7]:
yml_file_name = 'dcp_test_tmplate.yml'
yml_dir = os.path.join(tools_base_dir, 'data/run_files')
run_parameters = get_samples_clustering_run_parameters(yml_dir, yml_file_name)

run_parameters['spreadsheet_data_dir'] = '/Users/lanier4/pipeline_spreadsheets/aws_v3/SC_G_spreadsheets'
run_parameters['pheno_data_dir'] = '/Users/lanier4/pipeline_spreadsheets/aws_v3/SC_P_spreadsheets'

for k in sorted(run_parameters.keys()):
    print(k,':\t',run_parameters[k])

pheno_data_dir :	 /Users/lanier4/pipeline_spreadsheets/aws_v3/SC_P_spreadsheets
phenotype_full_path :	 ../data/spreadsheets/TEST_1_phenotype.tsv
pipeline_type :	 samples_clustering_pipeline
redis_credential :	 {'password': 'KnowEnG', 'host': 'knowhub.org', 'port': 6379}
results_directory :	 /Users/lanier4/dlanier_KnowEnG/keg_test_tools/notebooks/run_dir1488400151391120
run_directory :	 /Users/lanier4/dlanier_KnowEnG/keg_test_tools/data/run_files
run_file :	 dcp_test_tmplate.yml
source_hint :	 
spreadsheet_data_dir :	 /Users/lanier4/pipeline_spreadsheets/aws_v3/SC_G_spreadsheets
spreadsheet_name_full_path :	 ../data/spreadsheets/TEST_1_gene_expression.tsv
taxonid :	 9606


In [8]:
test_result_df = test_samples_clustering_cleanup(run_parameters)

Hsap.ccle.P.cyto_ic50.pos.c.df Hsap.ccle.G.gene_mut.binary.a.df
Hsap.ccle.P.cyto_ic50.pos.c.df Hsap.ccle.G.lg2_cn_ratio.real.a.df
Hsap.ccle.P.cyto_ic50.pos.c.df Hsap.ccle.G.qnorm_probe.pos.a.df
Hsap.dream11.P.cell_line_info.mixed.c.df Hsap.dream11.G.exome_mut.binary.a.df
Hsap.dream11.P.cell_line_info.mixed.c.df Hsap.dream11.G.gene_cn.pos.a.df
Hsap.dream11.P.cell_line_info.mixed.c.df Hsap.dream11.G.probe_expr.pos.a.df
Hsap.dream7_c1.P.cyto_gi50.pos.c.df Hsap.dream7_c1.G.lg2_cnv.real.a.df
Hsap.dream7_c1.P.cyto_gi50.pos.c.df Hsap.dream7_c1.G.lg2_probe.pos.a.df
Hsap.dream7_c1.P.cyto_gi50.pos.c.df Hsap.dream7_c1.G.methyl_prop.pos.a.df
Hsap.dream7_c1.P.cyto_gi50.pos.c.df Hsap.dream7_c1.G.rnaseq_expressed.binary.a.df
Hsap.dream7_c1.P.cyto_gi50.pos.c.df Hsap.dream7_c1.G.rnaseq_fpkm.pos.a.df
Hsap.gdsc.P.norm_ic50.real.c.df Hsap.gdsc.G.probe_expr.pos.a.df
Hsap.mayo_lcl.P.cyto_ec50.pos.c.df Hsap.mayo_lcl.G.med_lg2_norm_probe.real.a.df
Hsap.mayo_lcl.P.cyto_ec50.pos.c.df Hsap.mayo_lcl.G.norm_reg_sn

In [9]:
run_cleanup = False
test_result_df = dcp_test.test_data_cleanup(run_parameters, run_cleanup)


	Start testing samples_clustering_pipeline at Wed Mar  1 14:29:15 2017

 Hsap.ccle.P.cyto_ic50.pos.c.df
	 Hsap.ccle.G.gene_mut.binary.a.df

 Hsap.ccle.P.cyto_ic50.pos.c.df
	 Hsap.ccle.G.lg2_cn_ratio.real.a.df

 Hsap.ccle.P.cyto_ic50.pos.c.df
	 Hsap.ccle.G.qnorm_probe.pos.a.df

 Hsap.dream11.P.cell_line_info.mixed.c.df
	 Hsap.dream11.G.exome_mut.binary.a.df

 Hsap.dream11.P.cell_line_info.mixed.c.df
	 Hsap.dream11.G.gene_cn.pos.a.df

 Hsap.dream11.P.cell_line_info.mixed.c.df
	 Hsap.dream11.G.probe_expr.pos.a.df

 Hsap.dream7_c1.P.cyto_gi50.pos.c.df
	 Hsap.dream7_c1.G.lg2_cnv.real.a.df

 Hsap.dream7_c1.P.cyto_gi50.pos.c.df
	 Hsap.dream7_c1.G.lg2_probe.pos.a.df

 Hsap.dream7_c1.P.cyto_gi50.pos.c.df
	 Hsap.dream7_c1.G.methyl_prop.pos.a.df

 Hsap.dream7_c1.P.cyto_gi50.pos.c.df
	 Hsap.dream7_c1.G.rnaseq_expressed.binary.a.df

 Hsap.dream7_c1.P.cyto_gi50.pos.c.df
	 Hsap.dream7_c1.G.rnaseq_fpkm.pos.a.df

 Hsap.gdsc.P.norm_ic50.real.c.df
	 Hsap.gdsc.G.probe_expr.pos.a.df

 Hsap.mayo_lcl.P.cyto

In [10]:
test_result_df

,phenotype_file,validation_flag,message,spreadsheet_rows,spreadsheet_cols,cleanup_time
Dmel.bdgp.G.insitu_domain.binary.a.df,0,0,0,0.0,0.0,0.000000e+00
Hsap.ccle.G.gene_mut.binary.a.df,Hsap.ccle.P.cyto_ic50.pos.c.df,False,Failed to finish,0.0,0.0,0.000000e+00
Hsap.ccle.G.lg2_cn_ratio.real.a.df,Hsap.ccle.P.cyto_ic50.pos.c.df,False,Failed to finish,0.0,0.0,0.000000e+00
Hsap.ccle.G.qnorm_probe.pos.a.df,Hsap.ccle.P.cyto_ic50.pos.c.df,False,Failed to finish,0.0,0.0,0.000000e+00
Hsap.dream11.G.exome_mut.binary.a.df,Hsap.dream11.P.cell_line_info.mixed.c.df,False,Failed to finish,0.0,0.0,9.536743e-07
Hsap.dream11.G.gene_cn.pos.a.df,Hsap.dream11.P.cell_line_info.mixed.c.df,False,Failed to finish,0.0,0.0,0.000000e+00
Hsap.dream11.G.probe_expr.pos.a.df,Hsap.dream11.P.cell_line_info.mixed.c.df,False,Failed to finish,0.0,0.0,0.000000e+00
Hsap.dream7_c1.G.lg2_cnv.real.a.df,Hsap.dream7_c1.P.cyto_gi50.pos.c.df,False,Failed to finish,0.0,0.0,0.000000e+00
Hsap.dream7_c1.G.lg2_probe.pos.a.df,Hsap.dream7_c1.P.cyto_gi50.pos.c.df,False,Failed to finish,0.0,0.0,0.000000e+00
Hsap.dream7_c1.G.methyl_prop.pos.a.df,Hsap.dream7_c1.P.cyto_gi50.pos.c.df,False,Failed to finish,0.0,0.0,0.000000e+00
